In [1]:
import random,timeit
from sklearn import preprocessing
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from copy import deepcopy as dc
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection as ms

In [2]:
def random_search(n,dim):
    gens=[[0 if g != j else 1 for g in range(n)] for j in range(dim)]
    return gens



In [3]:
def evaluate(train_d,train_l,gen):
        mask=np.array(gen) > 0
        al_data=np.array([al[mask] for al in train_d])
        kf = ms.KFold(n_splits=4)
        s = 0
        for tr_ix,te_ix in kf.split(al_data):
            s+= RandomForestClassifier(n_estimators=25).fit(al_data[tr_ix],train_l[tr_ix]).score(al_data[te_ix],train_l[te_ix])#.predict(al_test_data)
        s/=4
        return s

In [4]:
def bees_optimization(bee,binary,i):
    binary=list(binary)
    j=random.randint(0,len(binary)-1)
    k=random.randint(0,len(binary)-1)
    while k==i:
        k=random.randint(0,len(binary)-1)
    fit=binary[j]+random.uniform(-1,1)*(binary[j]-binary[k])
    for x in range(bee):
        y=random.randint(0,len(binary)-1)
        while y==i:
            y=random.randint(0,len(binary)-1)
        r=random.uniform(0,1)
        if r<=fit:
            binary[y]=1
        
    return binary

In [5]:
def BABCO(train_d,train_l,n=10,max_iter=25,employee_percent=0.5,max_limit=5):
    """
    input:{ Eval_Func: Evaluate_Function, type is class
            n: Number of population, default=20
            max_iter: Number of max iteration, default=300
            }
    output:{
            Best position: type list(int) [1,0,0,1,.....]
            Nunber of 1s in best position: type int [0,1,1,0,1] → 3
            }
    """
    employed_bees = int(round(n*employee_percent))
    onlooker_bees = n - employed_bees       

    dim=len(train_d[0])
    global_best=float("-inf")
    global_position=tuple([0]*dim)
    gens_dict = {}
    limit=[0]*dim
    gens=random_search(dim,dim)
    for gen in gens:
        if tuple(gen) in gens_dict:
            score = gens_dict[tuple(gen)]
        else:
            score=evaluate(train_d,train_l,gen)
            gens_dict[tuple(gen)]=score
        if score > global_best:
            global_best=score
            global_position=dc(gen)
            
            
    for it in range(max_iter):
        for i in range(employed_bees):
            for i,x in enumerate(gens):
                gen=bees_optimization(employed_bees,x,i)
                if tuple(gen) in gens_dict:
                    score = gens_dict[tuple(gen)]
                else:
                    score=evaluate(train_d,train_l,gen)
                    gens_dict[tuple(gen)]=score

                if score > gens_dict[tuple(gens[i])]:
                    limit[i]=0
                    gens[i]= gen
                else:
                    limit[i]+=1

                if score > global_best:
                    global_best=score
                    global_position=dc(gen)

                if limit[i]>=max_limit:
                    gens[i]=[0 if g != i else 1 for g in range(dim)]
    
        for i in range(onlooker_bees):
            for i,x in enumerate(gens):
                gen=bees_optimization(employed_bees,x,i)
                if tuple(gen) in gens_dict:
                    score = gens_dict[tuple(gen)]
                else:
                    score=evaluate(train_d,train_l,gen)
                    gens_dict[tuple(gen)]=score

                if score > gens_dict[tuple(gens[i])]:
                    limit[i]=0
                    gens[i]= gen
                else:
                    limit[i]+=1

                if score > global_best:
                    global_best=score
                    global_position=dc(gen)

                if limit[i]>=max_limit:
                    gens[i]=[0 if g != i else 1 for g in range(dim)]

                
    return global_position,global_position.count(1)


In [6]:
def test_score(gen,tr_x,tr_y,te_x,te_y):
    mask=np.array(gen) == 1
    al_data=np.array(tr_x[:,mask])
    al_test_data=np.array(te_x[:,mask])
    return np.mean([RandomForestClassifier(n_estimators=25).fit(al_data,tr_y).score(al_test_data,te_y) for i in range(5)])

In [7]:
with open("/home/shubbham28/Downloads/Arrhythmia/heart.csv") as f:
    x=np.array([[float(d) for d  in data.split(',')] for data in f.read().splitlines()])
with open("/home/shubbham28/Downloads/Arrhythmia/heart_output.csv") as f:
    y=np.array([float(data) for data in f.read().splitlines()])
lab_enc = preprocessing.LabelEncoder()
training_scores_encoded = lab_enc.fit_transform(y)
train_d, test_d, train_l, test_l = train_test_split(x, training_scores_encoded, test_size=0.25)

In [8]:
k=[1 for r in range(len(x[0]))]
print test_score(k,train_d,train_l,test_d,test_l)
fattr=0
ftest=0.0
flist=[0 for r in range(len(x[0]))]
final_list=[0 for r in range(len(x[0]))]
start=timeit.default_timer()
for i in range(20):
    g,l=BABCO(train_d,train_l,n=10,max_iter=25,employee_percent=0.5,max_limit=5)
    fattr+=l
    test=test_score(g,train_d,train_l,test_d,test_l)
    ftest+=test
    for j in range(len(flist)):
        if g[j]==1:
            flist[j]+=1
    print("{0}  {1}  {2}  {3:.6f}".format(i+1,"".join(map(str,g)),l,test))
fattr=fattr/20
ftest=ftest/20
end=timeit.default_timer()
time=end-start
print flist,fattr
final=np.argsort(flist)[::-1][:fattr]
print final
for i in range(len(final)):
    final_list[final[i]]=1
print("{0}  {1}   {2}   {3:.6f}    {4:.4f}".format("Final: ","".join(map(str,final_list)),fattr,ftest,time))

0.8294117647058823
1  1111101101111  11  0.802941
2  0111110111111  11  0.817647
3  0110110011011  8  0.814706
4  0111111101111  11  0.835294
5  1011101101111  10  0.791176
6  0111100111111  10  0.829412
7  0011110001111  8  0.800000
8  0110111101111  10  0.820588
9  0111111011011  10  0.841176
10  0110100001011  6  0.835294
11  0111110011111  10  0.817647
12  0111111001111  10  0.800000
13  0110111001111  9  0.823529
14  0011111011111  10  0.838235
15  1111010101111  10  0.823529
16  0011101101111  9  0.811765
17  0000000000111  3  0.747059
18  0011111001111  9  0.829412
19  0110111111111  11  0.826471
20  0110110101011  8  0.820588
[3, 14, 19, 13, 18, 14, 11, 10, 7, 19, 16, 20, 20] 9
[12 11  9  2  4 10  5  1  3]
Final:   0111110001111   9   0.816324    3113.5976
